<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/STT%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [17]:
!pip install tfds-nightly
!pip install pydub

In [77]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import time

In [19]:
!git clone https://github.com/Jaidon-Smith/public_datasets.git

fatal: destination path 'public_datasets' already exists and is not an empty directory.


# Building the Model

In [20]:
!pip install --quiet tensorflow-text

import tensorflow_text as text
from tensorflow.python.platform import gfile

model_file = '/gdrive/MyDrive/Japanese/Bible/letters_v2.model'
model = gfile.GFile(model_file, 'rb').read()

tokenizer = text.SentencepieceTokenizer(model=model)

input_vocab_size = tokenizer.vocab_size().numpy()
print(input_vocab_size)

     |████████████████████████████████| 3.4MB 8.5MB/s 
13306


[This tutorial](https://www.tensorflow.org/tutorials/audio/simple_audio) has some good resources.

In [179]:
import pickle
jsut_speech_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_speech_lengths', "rb"))
jsut_text_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_text_lengths', "rb"))

In [180]:
tokenizer.tokenize(tf.constant("英語が分かりますか"))

<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([13305,  1383,   231,    16,   100,    21,    23,    12,    17,
          21], dtype=int32)>

In [181]:
#def preprocess_audio(old_audio):
#  audio = old_audio/tf.int16.max
#  audio = tf.cast(audio, tf.float32)
#  audio = tfio.audio.resample(audio, 48000, 24000)
#  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
#  return audio

In [182]:
import pykakasi
converter = pykakasi.kakasi()

In [208]:
def preprocess_text(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  return text

In [209]:
def preprocess_text_no_pad(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  #text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  text = tf.shape(text)[0]
  return text

In [210]:
# For loop that I don't think will work
#def preprocess_text(old_text):
#  batch_size = 
#  output = 
#  text = old_text.numpy().decode('utf-8')
#  text = tokenizer.tokenize(text)
#  text = tf.constant(text)
#  text = tf.pad(text, paddings=[[0 , 2000 - tf.shape(text)[0]]])
#  return text

In [211]:
#def preprocess_text_batches(old_text):
#  return tf.map_fn(
#    preprocess_text, old_text
#)

In [212]:
#text.numpy()[0].decode('utf-8')

In [213]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [214]:
#jsut_speech_lengths = {}

#for i, data in enumerate(dataset):
  
#  jsut_speech_lengths[data['id'].ref()] = data['speech'].shape[0]
#  break

In [215]:
def preprocess_audio(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  audio = tf.pad(audio, paddings=[[0 , 500 - tf.shape(audio)[0]], [0,0]])
  return audio

In [216]:
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  
  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  audio = tf.shape(audio)[0]
  return audio



```python
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)
  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  speech_lengths = tf.map_fn(lambda x: tf.shape(x)[0], audio, fn_output_signature=tf.int32)
  return speech_lengths
```



In [217]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='/gdrive/MyDrive/datasets/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [218]:
def convert_to_kana(text):
  kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
  return ''.join([i['hira'] for i in kana])

In [219]:
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": convert_to_kana(x["text"])})



```python
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech_lengths": tf.py_function(func=preprocess_audio_no_pad, inp=[x["speech"]],Tout=tf.int32),
                                 "speech": preprocess_audio(x["speech"]),
                                 "text": preprocess_text(x["text"])})
```



In [220]:
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"]),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"]),
                                 "text": preprocess_text(x["text"]),
                                 "text_lengths": preprocess_text_no_pad(x["text"])})

In [221]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [222]:
dataset = dataset.batch(8)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [199]:
#max = 0

#for i, data in enumerate(dataset):
#  value = data['speech'].shape[0]
#  if value > max:
#    max = value
#    print("new max is:", value, "with id:", data['id'].numpy())

In [200]:
ds = dataset.take(1)

In [201]:
for i in ds:
  data = i
  break

In [202]:
print(data["speech_lengths"])

tf.Tensor([222], shape=(1,), dtype=int32)


In [203]:
print(tf.shape(data['text']).numpy())

[  1 200]


In [204]:
print(data['text'][0][:100])

tf.Tensor(
[  35  318    7   22   24   15   33 1051   20   57   31  213  206   39
    7    3    6   17   15   22   20   77   33    1    5   33    8   13
   15    4    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int32)


In [43]:
#input_vocab_size = tokenizer.vocab_size() - 1
input_vocab_size = 2000
print(input_vocab_size)

2000


In [44]:
speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']

In [45]:
text_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([61, 46, 27, 45, 18, 69, 25, 26], dtype=int32)>

In [46]:
speech_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([222, 183,  99, 169,  69, 246, 100, 108], dtype=int32)>

# Using Transformer Encoder Blocks

In [61]:
d_model = 512

In [47]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [48]:
def scaled_dot_product_attention(q, k, v):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)


  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [49]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [50]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [51]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training):

    attn_output, _ = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [90]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(d_model)

    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training=True):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # Makes input larger by a constant
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training)
    
    return x  # (batch_size, input_seq_len, d_model)

In [59]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [62]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [63]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [91]:
transformer = tf.keras.models.Sequential([
 
  Encoder(num_layers=6,
          d_model=d_model,
          num_heads=8,
          dff=2048, 
          maximum_position_encoding=10000),
 
  tf.keras.layers.Dense(input_vocab_size)
])

In [72]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [92]:
transformer(speech)

<tf.Tensor: shape=(8, 500, 2000), dtype=float32, numpy=
array([[[-0.23670131,  0.5035986 , -0.9696934 , ..., -0.32356334,
         -0.15642008,  0.52041733],
        [-0.20735277,  0.04722196, -0.7190511 , ...,  0.11990781,
         -0.16057464,  0.84492683],
        [-0.40871394,  0.49407247, -0.8547423 , ...,  0.5248531 ,
          0.26138884,  0.70770925],
        ...,
        [-0.6523657 ,  0.62752515, -1.0001107 , ..., -0.03352382,
          0.5254311 ,  0.93971825],
        [ 0.24591252,  0.28143832, -0.72985864, ...,  0.2522912 ,
          0.13842534,  1.1001192 ],
        [-0.24758309,  0.37546644, -0.4372609 , ...,  0.19912373,
         -0.05195087,  0.9075059 ]],

       [[-0.04687387,  0.8616662 , -0.72437745, ...,  0.87763244,
          0.66539514,  0.42403227],
        [ 0.04295494,  0.01894596,  0.05669194, ...,  0.29154584,
          0.2664561 ,  0.11589174],
        [-0.20177966,  0.8460218 , -0.57802176, ...,  0.494439  ,
          0.25340444,  0.18832803],
        ...

In [224]:
@tf.function
def train_step(speech, text, speech_lengths, text_lengths):
  with tf.GradientTape() as tape:
    logits = transformer(speech)

    labels = text
    label_length = text_lengths
    logit_length = speech_lengths
    unique = tf.nn.ctc_unique_labels(text)


    loss = tf.nn.ctc_loss(
        labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
    )

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)

In [225]:
EPOCHS = 5

In [226]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()

  # inp -> portuguese, tar -> english
  for (batch, data) in enumerate(dataset):
    #print(batch)
    #if batch == 9:
    #  print(tokenizer.detokenize(data['text']).numpy()[0].decode('utf-8'))
    #  print(data['text'].numpy()[0])
    #  continue

    train_step(data['speech'], data['text'], data['speech_lengths'], data['text_lengths'])

    if batch % 1 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f}')

  break
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print (f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


Epoch 1 Batch 0 Loss 174.3032
Epoch 1 Batch 1 Loss 174.5954
Epoch 1 Batch 2 Loss 173.0407
Epoch 1 Batch 3 Loss 173.3390
Epoch 1 Batch 4 Loss 171.6476
Epoch 1 Batch 5 Loss 171.2476
Epoch 1 Batch 6 Loss 170.6131
Epoch 1 Batch 7 Loss 169.4121
Epoch 1 Batch 8 Loss 168.5527
Epoch 1 Batch 9 Loss 167.4898
Epoch 1 Batch 10 Loss 166.2850
Epoch 1 Batch 11 Loss 165.1365
Epoch 1 Batch 12 Loss 164.1532
Epoch 1 Batch 13 Loss 163.0261
Epoch 1 Batch 14 Loss 162.3440
Epoch 1 Batch 15 Loss 161.8528
Epoch 1 Batch 16 Loss 160.9754
Epoch 1 Batch 17 Loss 160.1003
Epoch 1 Batch 18 Loss 159.4562
Epoch 1 Batch 19 Loss 158.6361
Epoch 1 Batch 20 Loss 157.8520
Epoch 1 Batch 21 Loss 157.1543
Epoch 1 Batch 22 Loss 156.2693
Epoch 1 Batch 23 Loss 155.9056
Epoch 1 Batch 24 Loss 155.1221
Epoch 1 Batch 25 Loss 154.5388
Epoch 1 Batch 26 Loss 153.9061
Epoch 1 Batch 27 Loss 153.1711
Epoch 1 Batch 28 Loss 152.6128
Epoch 1 Batch 29 Loss 152.1205
Epoch 1 Batch 30 Loss 151.5712
Epoch 1 Batch 31 Loss 151.0845
Epoch 1 Batch 32 L

KeyboardInterrupt: ignored

In [ ]:
#

## CTC Loss



```python
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')
```



In [53]:
#dense1 = tf.keras.layers.Dense(input_vocab_size)

In [54]:
dense1 = tf.keras.layers.Dense(2000)

In [55]:
dense1_speech = dense1(speech)

In [56]:
dense1_speech.shape

TensorShape([8, 500, 2000])



```python
text_lengths = tf.map_fn(lambda x: jsut_text_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)

speech_lengths = tf.map_fn(lambda x: jsut_speech_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)
```



In [57]:
labels = text
logits = dense1_speech
label_length = text_lengths
logit_length = speech_lengths
unique = tf.nn.ctc_unique_labels(text)


loss = tf.nn.ctc_loss(
    labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
)

Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


In [58]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8213451224441148177, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5724101691350584524
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [164]:
!pip install pykakasi

     |████████████████████████████████| 2.4MB 8.0MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 


In [176]:
import pykakasi
converter = pykakasi.kakasi()

In [175]:
kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
''.join([i['hira'] for i in kana])

'えぞにこもるきゅうばくふぐんにたいするこうげきのしきをとる。'

In [169]:
kks.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')

[{'hepburn': 'ezo',
  'hira': 'えぞ',
  'kana': 'エゾ',
  'kunrei': 'ezo',
  'orig': '蝦夷',
  'passport': 'ezo'},
 {'hepburn': 'ni',
  'hira': 'に',
  'kana': 'ニ',
  'kunrei': 'ni',
  'orig': 'に',
  'passport': 'ni'},
 {'hepburn': 'komo',
  'hira': 'こも',
  'kana': 'コモ',
  'kunrei': 'komo',
  'orig': '籠も',
  'passport': 'komo'},
 {'hepburn': 'ru',
  'hira': 'る',
  'kana': 'ル',
  'kunrei': 'ru',
  'orig': 'る',
  'passport': 'ru'},
 {'hepburn': 'kyuu',
  'hira': 'きゅう',
  'kana': 'キュウ',
  'kunrei': 'kyuu',
  'orig': '旧',
  'passport': 'kyuu'},
 {'hepburn': 'bakufu',
  'hira': 'ばくふ',
  'kana': 'バクフ',
  'kunrei': 'bakufu',
  'orig': '幕府',
  'passport': 'bakufu'},
 {'hepburn': 'gun',
  'hira': 'ぐん',
  'kana': 'グン',
  'kunrei': 'gun',
  'orig': '軍',
  'passport': 'gun'},
 {'hepburn': 'ni',
  'hira': 'に',
  'kana': 'ニ',
  'kunrei': 'ni',
  'orig': 'に',
  'passport': 'ni'},
 {'hepburn': 'taisuru',
  'hira': 'たいする',
  'kana': 'タイスル',
  'kunrei': 'taisuru',
  'orig': '対する',
  'passport': 'taisuru'},
 {'